# Prepare synthetic data 
It is going to be much easier to develop if you have realistic synthetic data.
Here we take a sql query that generates a single tabular output.
We run that query against the live identifiable data once.
We then use the [Synthetic Data Vault](https://sdv.dev/SDV/index.html) to prepare a synthetic model of those data.
The code below serves as a vignette for that process but will need adjusting to match the exact contents of the original query.

More complex examples that include multiple tables with joins and dependencies are also possible.

This notebook should be run interactively just once

## Set-up, query and return the data as datafame
The query lives in `./src/api/sitrep/` where `.` represents the project root.
If you run this JupyterNotebook using the local **Makefile** and `make run` then that query will be automatically copied here.

So first steps should be (from _this_ directory)
```sh
make mock1build
make mock2copyin
make mock3run
```
then navigate to http://uclvlddpragae07:8091/lab/tree/steve/work/synth_test_data_sitrep.ipynb

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine

Now rather than using a SQL query, this time we are going to use an existing API to populate a dataframe

In [ ]:
response = requests.get("http://uclvlddpragae07:5006/icu/live/T03/ui")
df = pd.DataFrame.from_dict(response.json()["data"])

In [ ]:
df.dtypes

In [ ]:
df.head()

## Generate a synthetic version of the real data

Use the table above to generate the metadata you need for the synthetic data

https://sdv.dev/SDV/user_guides/relational/relational_metadata.html#relational-metadata

In [ ]:
# Minimal imports
from sdv import SDV, Metadata

SDV does not handle timezones nicely so remove

In [ ]:
# PostgreSQL returns datetimes with tz info which sdv does not seem to be able to handle
def remove_timezone(df, col_name: str) -> pd.DataFrame:
    """sdv does not like timezones"""
    df[col_name] = pd.to_datetime(df[col_name], utc=True).dt.tz_localize(None)
    return df

In [ ]:
tz_cols = [
    "admission_dt",
]
for col in tz_cols:
    print(col)
    df = remove_timezone(df, col)

### Define PII that must be faked and not modelled

Define fields that contain PII and need faking (see the sketchy documentation [here](https://sdv.dev/SDV/developer_guides/sdv/metadata.html?highlight=pii#categorical-fields-data-anonymization) and the [Faker Documentation](https://faker.readthedocs.io/en/master/providers.html) for a full list of providers. Here is a brief example that specifies Fakers for [name](https://faker.readthedocs.io/en/master/providers/faker.providers.person.html#faker.providers.person.Provider.name) and [date of birth](https://faker.readthedocs.io/en/master/providers/faker.providers.date_time.html#faker.providers.date_time.Provider.date_of_birth). Note that you must pass arguments to a faker as a list.

In [ ]:
fields = {
    "dob": {
        "type": "datetime",
        "format": "%Y-%m-%d",
        "pii": True,
        "pii_category": "date_of_birth",
    },
    "name": {"type": "categorical", "pii": True, "pii_category": "name"},
}

NB: sdv also doesn't always recognise the columns correctly. Here we specify data_of_birth explicitly as a date whilst working on the larger task of defining columns that contain PII. See [field details](https://sdv.dev/SDV/developer_guides/sdv/metadata.html#field-details)

Now a full specification for the Sitrep data

In [ ]:
fields = {
    "dob": {
        "type": "datetime",
        "format": "%Y-%m-%d",
        "pii": True,
        # the 'pii_category' key defines the Faker function name (method)
        "pii_category": "date_of_birth",
    },
    "admission_age_years": {
        "type": "numerical",
        "pii": True,
        "pii_category": ["random_number", 2],
    },
    "name": {"type": "categorical", "pii": True, "pii_category": "name"},
    "mrn": {"type": "categorical", "pii": True, "pii_category": ["random_number", 8]},
    "csn": {
        "type": "categorical",
        "pii": True,
        "pii_category": ["numerify", "10########"],
    },
}

Prepare the metadata

In [ ]:
metadata = Metadata()
metadata.add_table(
    name="tabpid",
    data=df,
    fields_metadata=fields,
)

In [ ]:
# Inspect the conversion that metadata.add_table did to the dataframe that you loaded
metadata.get_table_meta("tabpid")

Prepare the table(s)

In [ ]:
tables = dict(tabpid=df)

Fit the model

In [ ]:
sdv = SDV()
sdv.fit(metadata, tables)

Inspect the original data

In [ ]:
df.to_csv("sitrep_orig.csv")

Inspect the synthetic data

In [ ]:
sdv.sample_all()["tabpid"].head()

Finally transform admission_age_years to match the fake DoB

In [ ]:
df = sdv.sample_all()["tabpid"]

In [ ]:
df["admission_age_years"] = (
    np.floor(((df["admission_dt"] - df["dob"]) / np.timedelta64(1, "Y")))
).astype(int)

In [ ]:
df.head()

In [ ]:
df.to_csv("sitrep_synth.csv")

### Save the synthetic data

Options
- save the model and not the synthetic data (but then you need *sdv* to run the model)
- save the data (need some care with type conversions if you use csv etc.)

In [ ]:
sdv.save("mock_model.pkl")

In [ ]:
df.to_hdf("mock_sitrep.h5", "data")

In [ ]:
pd.read_hdf("mock_sitrep.h5", "data")